In [ ]:
! pip install glaredb --upgrade

In [ ]:
import glaredb

In [ ]:
con = glaredb.connect()

In [ ]:
con.execute(
    f"""   
        SELECT * FROM read_parquet(
            's3://glaredb-sandbox/769265408ff247babfa1f87baee4ef22-0.parquet',
            access_key_id => '{aws_access_key_id}',
            secret_access_key => '{aws_secret_access_key}',
            region=>'us-east-2'
        );
    """
).to_pandas()

In [ ]:
con.execute(f"""
    SELECT * FROM 
    read_snowflake(
    '{snowflake_account}.us-central1.gcp', 
    '{snowflake_username}',
    '{snowflake_password}',
    'sandbox',
    'compute_wh',
    'accountadmin',
    'public',
    'nyc_sales'
    ) snowflake 
""").to_pandas()

In [ ]:
df = con.sql(
    f"""
        SELECT snowflake_nyc_sales.bin,
                snowflake_nyc_sales.address,
                snowflake_nyc_sales.zip_code,
                snowflake_nyc_sales.sale_date,
                snowflake_nyc_sales.sale_price,
                COUNT (distinct s3_tree.spc_latin) number_tree_species,
                COUNT (s3_tree.spc_latin) number_trees
        FROM
            read_snowflake(
                '{snowflake_account}.us-central1.gcp', 
                '{snowflake_username}',
                '{snowflake_password}',
                'sandbox',
                'compute_wh',
                'accountadmin',
                'public',
                'nyc_sales'
            ) snowflake_nyc_sales
        LEFT JOIN 
            read_parquet(
                's3://glaredb-sandbox/769265408ff247babfa1f87baee4ef22-0.parquet',
                access_key_id => '{aws_access_key_id}',
                secret_access_key => '{aws_secret_access_key}',
                region=>'us-east-2'
            ) s3_tree
        ON snowflake_nyc_sales.bin = s3_tree.bin
        WHERE snowflake_nyc_sales.borough = 3
        GROUP BY snowflake_nyc_sales.bin,
                snowflake_nyc_sales.address,
                snowflake_nyc_sales.zip_code,
                snowflake_nyc_sales.sale_date,
                snowflake_nyc_sales.sale_price
        ORDER BY number_trees DESC
    """).to_pandas()

df

In [ ]:
con.execute("""
    SELECT count(*) FROM df WHERE number_trees > 0
""").to_pandas()